**Libraries**

In [ ]:
#load data
import h5py
with h5py.File(r'your_directory\usps.h5', 'r') as hf:
        train = hf.get('train')
        X_tr = train.get('data')[:]
        y_tr = train.get('target')[:]
        test = hf.get('test')
        X_te = test.get('data')[:]
        y_te = test.get('target')[:]

In [ ]:
import numpy as np
import pandas as pd
import random
from copy import deepcopy

In [ ]:
df=pd.DataFrame(np.concatenate((np.column_stack((X_tr,y_tr)),np.column_stack((X_te,y_te))), axis = 0))
df

**Auxiliary functions**

In [ ]:
#FUNCTION for changing classificator
def chg_pred(df,a):
    bin_y = np.where(df.loc[:,256] == a, 1, -1)
    df[257] = bin_y

In [ ]:
def sign(x): #sign function
    if x > 0:
        return 1
    elif x == 0:
        return 0
    else:
        return -1

In [ ]:
#define the gaussian kernel function
def gauss_k(gamma,a,b):
    return np.exp((-1/2*gamma)*(np.linalg.norm((a-b), ord = 1)**2))

In [ ]:
def assign_pred(df): #takes as input result arrays for the ten predictors (otp_train|otp_test)
    return pd.DataFrame(df).T.idxmax(axis=1)

In [ ]:
def comp_mc(df1,df2): #takes as input the training/test folds and the result of assign_pred and compute the average mc error
    comp = df1.loc[:,256].reset_index(drop=True) == df2
    errors_count = comp.value_counts().get(False, 0)
    total_count = len(comp)
    return errors_count/total_count #fraction of examples incorrectly classified

**PEGASOS**

In [ ]:
#train pegasos
def train_pegasos(df_train,T,lam):
    t = 1
    S = [] #list for examples classified incorrectly
             
    for i in range(T):
        eta = 1/(lam*t) #update eta for the current iteration
        sample = df_train.iloc[np.random.choice(len(df_train), replace=True, size = 1)] #draw an example
        x_t = sample.iloc[0,0:256]
        y_t = sample.iloc[0,257]
        
        g_list = [y_s * gauss_k(0.25, x_s, x_t) for y_s, x_s in S] #note that we are using bi-linearity of the inner product
        g = eta * sum(g_list) #compute g using current eta factor    
                
        if y_t * g < 1: #check condition
            S.append((y_t,x_t))    
     
        t += 1 #update counter
    
    return S
#The algorithm returns the support vectors' set , used as input in the prediction

In [ ]:
def pred_pegasos(df, S):
    pred_vec = np.repeat(0, len(df[256]))
    
    for i in range(len(df)):
        x_t = df.iloc[i,0:256]
        g_list = [y_s * gauss_k(0.25, x_s, x_t) for y_s, x_s in S]
        g = sum(g_list)
        
        pred_vec[i] = sign(g)
    
    return pred_vec

**CV**

In [ ]:
lambdas = [1*10**-10, 1*10**-5, 1*10**-3, 1*10**-1, 1*10**5]
iterations = [250, 500, 1000, 3500, 7438] 

S_h = list(np.repeat(0, 10)) #container for storing different sets of support vectors (one for each predictor h)
otp_train = list(np.repeat(0, 10)) #lists for storing vectors of predictions for different predictors
otp_test= list(np.repeat(0, 10))                                   
cv_error = list(np.repeat(0, 10)) #lists for storing errors and results of cv 
cv_results = list(np.repeat(0, 9))

df = df.sample(frac=1) #reshuffle
folds = np.array_split(df, 5) #create folds

indx = 0

for lam in lambdas:
    for T in iterations:
        for f in range(5):
            df_test = folds.pop(0) #take first element
            df_test = df_test.reset_index(drop=True) #rearrange indexes
            df_train = pd.DataFrame(np.concatenate((folds),axis = 0)) #concatenate other elements          
                
            for h in range(10):
                df_train_copy = deepcopy(df_train)
                df_test_copy = deepcopy(df_test)
                chg_pred(df_train_copy, h)
                chg_pred(df_test_copy, h)
                
                S_h[h] = (train_pegasos(df_train_copy,T,lam)) #train current predictor
                otp_train[h] = pred_pegasos(df_train_copy, S_h[h]) #classify train with current predictor
                otp_test[h] = pred_pegasos(df_test_copy, S_h[h]) #classify test with current predictor
        
            cv_error[f] = comp_mc(df_train_copy,assign_pred(otp_train)) #first five elements of cv_error are avg. fth fold's train error
            cv_error[f+5] = comp_mc(df_test_copy,assign_pred(otp_test)) #second five elements of cv_error are avg. fth fold's test error
    
            folds.append(df_test) #append test fold in last position
    
        cv_results[indx] = (sum(cv_error[0:5])/5, sum(cv_error[5:10])/5)
        indx += 1
    

**Data visualization**

In [ ]:
df = pd.DataFrame(cv_results)
test_errors = df[1] 
train_errors = df[0]

test= { 'lambda' : np.repeat([1*10**-10, 1*10**-5, 1*10**-3, 1*10**-1, 1*10**5], 5),
     'Iterations' : [int(y) for y in [str(x) for x in [250, 500, 1000, 3500, 7438]]*5],
     'Test_CV_error' : test_errors }
test=pd.DataFrame(test)

train= { 'lambda' : np.repeat([1*10**-10, 1*10**-5, 1*10**-3, 1*10**-1, 1*10**5], 5),
     'Iterations' : [int(y) for y in [str(x) for x in [250, 500, 1000, 3500, 7438]]*5],
     'Training_CV_error' : train_errors }
train=pd.DataFrame(train)

train=train.pivot(index='lambda',columns='Iterations',values='Training_CV_error')
test=test.pivot(index='lambda',columns='Iterations',values='Test_CV_error')


In [ ]:
from IPython.display import display_html
import seaborn as sns

train_styler = train.style.set_table_attributes("style='display:inline'").set_caption('Training CV errors').format_index(formatter = lambda x: '%.2e' % x ).background_gradient(cmap= "hot")
test_styler = test.style.set_table_attributes("style='display:inline'").set_caption('Test CV Errors').format_index(formatter = lambda x: '%.2e' % x ).background_gradient(cmap= "hot")

display_html(train_styler._repr_html_() + test_styler._repr_html_(), raw=True)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
sns.heatmap(pd.DataFrame(test), annot = True, fmt='g', cmap= "hot", )
plt.title('Test CV Errors', fontsize = 12)